# Usage example on model of _Corynebacterium tuberculostearicum (high GC Gram+)_.
In this notebook we show how to use the Mass Charge Curation python package. As an example model we use a model of [_Corynebacterium tuberculostearicum (high GC Gram+)_, strain DSM 44922](https://www.ncbi.nlm.nih.gov/assembly/GCF_013408445.1/) created with [CarveMe Version 1.5.1](https://carveme.readthedocs.io/en/latest/index.html), which here is simply called _model.xml_.

If you are interested in a more verbose output you can uncomment the following line:

In [1]:
#import logging; logging.basicConfig(format='%(levelname)s: %(filename)s %(lineno)d, %(funcName)s: %(message)s', level=logging.INFO) 

## Dependencies
First we will check if all dependencies of the curation package are fullfilled.

In [2]:
try: import numpy
except Exception as e: print("You seem to be missing numpy. You can usually install it via 'pip install numpy'"); raise e
try: import pandas
except Exception as e: print("You seem to be missing pandas. You can usually install it via 'pip install pandas'"); raise e
try: import requests
except Exception as e: print("You seem to be missing requests. You can usually install it via 'pip install requests'"); raise e
try: import matplotlib
except Exception as e: print("You seem to be missing matplotlib. You can usually install it via 'pip install matplotlib'"); raise e
try: import z3
except Exception as e: print("You seem to be missing z3 or the corresponding python bindings. See https://github.com/Z3Prover/z3 on how to install it."); raise e

Next we see if the Mass Charge Curation package is installed properly.

In [3]:
try: import MCC
except Exception as e: print("The mass charge curation package does not seem to be installed correctly. Make sure you have the correct python version installed and try running pip install -e ./.. in the folder of this notebook."); raise e

## Setting input path
We can use the package with different type of models. Depending on what else we intend to do with the model we can already read it in using libsbml or cobraPy or immediatly pass in the path of the model (This will cause the module to read it in itself using libsbml).

In [4]:
model = "model.xml"

# Alternatives:
# reading in the model beforehand using libsbml:
'''
import libsbml
reader = libsbml.SBMLReader()
document = reader.readSBML(model)
model = document.getModel()
'''
# reading in the model beforehand using cobraPy:
'''
import cobra
model = cobra.io.read_sbml_model(model)
'''
print(model)

model.xml


## Basic Model Information
First we can take a quick look at the model using the module ModelInterface class:

In [5]:
mi = MCC.ModelInterface(model)
print(f"The model has {len(mi.metabolites)} metabolites and {len(mi.reactions)} reactions.")
unbalanced_reactions = [r_id for r_id, r in mi.reactions.items() if (not r.is_balanced())]
print(f"There are {len(unbalanced_reactions)} unbalanced reactions with {len(mi.get_pseudo_reactions())} pseudo reactions.")

The model has 1019 metabolites and 1481 reactions.
There are 446 unbalanced reactions with 196 pseudo reactions.


## Curating Mass and Charge
We can now instantiate a curation class. There are different ways to use the package, depending how data should be gathered, how much curation has already been done and how much data is available offline.

We will first give an example of the most simple usage, downloading as many databases as possible and updating all [identifiers.org](https://identifiers.org/) identifiers we need for the package to work optimally. This will take significantly longer (**~ 15 Minutes**) than running the algorithm on an already annotated model, however it is important to use the most up-to-date identifiers if we want to include as much information as possible.

This will create a folder _/data_ in the current directory where all database information is downloaded to.

The arguments are as follows:
* **model**: Model we want to curate.
* **data_path**: Path to the directory containing database files. Defaults to _/data_. If the directory does not exist, it will be created. If a file cannot be found, we will try to download it. 
* **update_ids**: If this is set to _True_, we will first try to update all [identifiers.org](https://identifiers.org/) ids. This will take a while but is important to properly index the different databases. Defaults to _False_. 

**Note**: It is expected to see _No objective coefficients in model. Unclear what should be optimized_ warnings here, this poses no problem for this package.

In [6]:
balancer = MCC.MassChargeCuration(model = model, data_path = "./data", run_optimization = True, update_ids = False)

KeyboardInterrupt: 

## Evaluating the result
To see the results at a first glance, we can generate a quick visual report, which tells us how many reactions are balanced now, and how many assignments were changed.

In [ ]:
balancer.generate_visual_report().show()

### Metabolite Report
The algorithm should be able to give a reason for every assignment that it chooses. We can have a look at these reasons in the metabolite report of the balancer.

The report holds valuable information how the algorithm decided which assignment to choose and can be useful during further manual curation. The entries of the resulting DataFrame are the following:
* **Id**: Id of the metabolite in the model.
* **Name**: Name of the metabolite in the model.
* **Determined Formula**: Formula that was assigned by the algorithm.
* **Determined Charge**: Charge that was assigned by the algorithm.
* **Previous Formula**: Formula that was assigned before the algorithm.
* **Previous Charge**: Charge that was assigned before the algorithm.
* **Inferrence Type**: Category of how the assignment was determined.
    - Unconstrained: No information about the metabolite was found or only incomplete (wildcard containing) formulae were found and we could not find a concrete formula either. Should contain a wildcard.
    - Inferred: No information about the metabolite was found or only incomplete (wildcard containing formulae were found, however we arrived at a concrete formula. Should not contain a wildcard.
    - Clean: Information about the metabolite was found and used.
* **Reasoning**: Reasoning how the assignment was determined. Can contain:
    - database name:database identifier: The formula could be found in this database under this identifier.
    - (unconstrained) Target: This assignment was chosen because it is the same as in the original (target) model. Unconstrained means that the original (target) model seemed to be missing a wildcard symbol that was thus added.
    - Reaction_id (metabolite id -> Reasoning...): The assignment for this metabolite must follow from other reasons. The given reaction id and metabolite reasons make it so that if the model must be balanced, this metabolite must have the determined assignment.
    - Used Databases: The databases which back up the determined assignment.
    - Previous Databases: The databases which back up the previous assignment.

In [ ]:
metabolite_report_df = balancer.generate_metabolite_report()
metabolite_report_df[::200]

Usually we are interested in the assignments which differ from the original report. We can do this by indexing the Dataframe accordingly.

In [ ]:
# If you like to see the entire report, uncomment the following line
#pandas.set_option("display.max_row", None)
metabolite_report_df[metabolite_report_df["Similarity"] != "Same"]

We might also be interested in all assignments which are not backed by a database:

In [ ]:
metabolite_report_df[metabolite_report_df["Used Databases"] == ""]

### Reaction Report
Finally, especially for further curation, we might be interested in the remaining imbalanced reactions. For this the algorithm can also provide a report.

The report also includes reactions which are technically balanced but where many protons had to be added to arrive at that result.

The fields are as follows:
* **Id**: Id of the reaction in the model.
* **Unbalanced Reaction**: Name of the reaction in the model and corresponding equation.
* **Unbalanced Type**: Type of imbalance. Can be both Mass and charge, only mass, only charge or high proton count.
* **Reason**: Set of reactions which caused the reaction to be imbalanced. This effectively means that these reactions could not be balanced together. The sets are minimal, but for example for BTS2, it would not help to remove HCYSMT, as BTS2 would require protons to have no charge and HCYSMT is only one of many reactions which then would not be balanced.
* **Shared Metabolites**: Metabolites which are shared between the reactions which are listed in Reason. Can give an indication where the problem might lie.
* **Mass Difference**: Mass imbalance.
* **Charge Difference**: Charge imbalance.

In [ ]:
balancer.generate_reaction_report()

### Writing Files
The mass charge curation writes directly to the model that was given to it. Thus, if we want to write our model, we can just pass the model to cobrapy. If you want to keep your old model, you should make sure to not overwrite it here.

In [ ]:
balancer.model_interface.write_model("model_MCC.xml")

For the reports, we can add filenames to the functions to write the visual report to a .png file and the metabolite and reaction DataFrames to .csv files.

In [ ]:
balancer.generate_visual_report(f"{model.id}_visual")
balancer.generate_metabolite_report(f"{model.id}_metabolites")
balancer.generate_reaction_report(f"{model.id}_reactions")
pass

## Fixing Assignments
For some metabolites we might want to fix their assignment, based on some knowledge that is not encoded in the used databases or is not used as it will yield a (theoretically) suboptimal result. We can do this by passing a dictionary mapping the metabolites id to the fixed assignment. If, for example, we want the metabolite with the id "M_h2o_c" to have the certain formula "CO2" and charge of 5 and the metabolite with the id "M_h_c" have a formula of "C5" and a charge of -7, we can tell this to our algorithm in the following way:

In [11]:
fixed_assignments = {"M_h2o_c" : ("CO2", 5), "M_h_c" : ("C5", -7)}
balancer = MCC.MassChargeCuration(model = model, data_path = "./data", run_optimization = True, update_ids = False, fixed_assignments = fixed_assignments)

In [12]:
unbalanced_reactions = [r_id for r_id, r in balancer.model_interface.reactions.items() if (not r.is_balanced())]
print(f"There are {len(unbalanced_reactions)} unbalanced reactions with {len(balancer.model_interface.get_pseudo_reactions())} pseudo reactions.")

There are 963 unbalanced reactions with 196 pseudo reactions.


In [13]:
balancer.model_interface.metabolites["M_h2o_c"]

Metabolite(id='M_h2o_c', name='H2O H2O', formula=CO2, charge=5, SBO=247)

In [14]:
balancer.model_interface.metabolites["M_h_c"]

Metabolite(id='M_h_c', name='H+', formula=C5, charge=-7, SBO=247)

We can see that setting strange values for water and hydrogen will make many reactions unbalancable but that the balancer adheres to the fixed values.

## Using BioCyc flat files
If you have access (currently requires an explicit academic license or a subscription) and downloaded the BioCyc flat files, you can pass the corresponding directory as well. Assuming we have already updated all the ids in the last step, we can set _update_ids_ to _False_.

The additional argument is:
* **biocyc_path**: Directory containing BioCyc flat files.

In [ ]:
balancer = MCC.MassChargeCuration(model = model, data_path = "./data", update_ids = True, biocyc_path = "./data/23.5/data")